In [2]:
import re

with open("gingatetsudono_yoru.txt", mode="r", encoding="utf-8") as f:  
    text_original = f.read()

text = re.sub("《[^》]+》", "", text_original) 
text = re.sub("［[^］]+］", "", text) 
text = re.sub("[｜ 　]", "", text)

print("Number of characters:", len(text)) 

Number of characters: 38753


In [3]:
# Num of time series
n_rnn = 10  
batch_size = 128
epochs = 60
# Num of neurons in hidden layer
n_mid = 128  

In [6]:
import numpy as np

chars = sorted(list(set(text)))  
print("Num of characters (no duplicate)", len(chars))
char_indices = {}  # Key: character, Value: index
for i, char in enumerate(chars):
    char_indices[char] = i
indices_char = {}  # Key: index, Value: character
for i, char in enumerate(chars):
    indices_char[i] = char
 
# Time series data and characters to be predicted
time_chars = []
next_chars = []
for i in range(0, len(text) - n_rnn):
    time_chars.append(text[i: i + n_rnn])
    next_chars.append(text[i + n_rnn])
 
# One-hot encoding
x = np.zeros((len(time_chars), n_rnn, len(chars)), dtype=np.bool)
t = np.zeros((len(time_chars), len(chars)), dtype=np.bool)
for i, t_cs in enumerate(time_chars):
    t[i, char_indices[next_chars[i]]] = 1  
    for j, char in enumerate(t_cs):
        x[i, j, char_indices[char]] = 1  
        
print("x.shape:", x.shape)
print("t.shape", t.shape)

Num of characters (no duplicate) 1049
x.shape: (38743, 10, 1049)
t.shape (38743, 1049)


In [7]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN

model = Sequential()
model.add(SimpleRNN(n_mid, input_shape=(n_rnn, len(chars))))
model.add(Dense(len(chars), activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer="adam")
print(model.summary())

Using TensorFlow backend.







_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 128)               150784    
_________________________________________________________________
dense_1 (Dense)              (None, 1049)              135321    
Total params: 286,105
Trainable params: 286,105
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
from keras.callbacks import LambdaCallback
 
def on_epoch_end(epoch, logs):
    print("epoch: ", epoch)

    beta = 5  # Coefficient for adjusting probability distribution
    prev_text = text[0:n_rnn]  # input
    created_text = prev_text  # text to be generated
    
    print("seed: ", created_text)

    for i in range(400):
        # Convert input to one-hot encoding
        x_pred = np.zeros((1, n_rnn, len(chars)))
        for j, char in enumerate(prev_text):
            x_pred[0, j, char_indices[char]] = 1
        
        # Predict and get next character
        y = model.predict(x_pred)
        p_power = y[0] ** beta  # Adjusting probability distribution
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power))        
        next_char = indices_char[next_index]

        created_text += next_char
        prev_text = prev_text[1:] + next_char

    print(created_text)
    print()

# Set a function to be executed at the end of the epoch. 
epock_end_callback= LambdaCallback(on_epoch_end=on_epoch_end)

In [10]:
history = model.fit(x, t,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[epock_end_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/60





38743/38743 [==============================] - 10s 265us/step - loss: 4.9656
epoch:  0
seed:  「ではみなさんは、そ
「ではみなさんは、そのた、のした。
なののした。のいのした。ましたした。してのは、いした。しにた。した。たした。たいしした。ししなたたりにした。ししした。のにののンした。っのしてのしたしたにいしたたしかのした。したいしたのした。にののいてたにいしいた、ました。しした。いいした。したいたしし、のたいたたしかいのいしたしたいたした。うの、た。し、なした。し、いののいしし。したかの、の、した。のいにのした。っしいししたのしいしのしてもしした。したしし。のりった。しといたのした。かいなのした。いのにるたにいしした。ののした。のいくのかした。いしったの、ししのののしのののした。ししのした。のいないたしのしたしたにのにのしてしかしたのたいしてたてのした。のしのしたのにししのたました。たいした。かいて、ののの、した。のしした。ました。のにの、い。ししてしたしたの、にののたののかのにいにした。いしたののした。いのののののした

Epoch 2/60
38743/38743 [==============================] - 10s 254us/step - loss: 4.4547
epoch:  1
seed:  「ではみなさんは、そ
「ではみなさんは、そっていました。
「のうののののののりのをっ拝景ニのした。
「のをはっていました。
「あののいの、うにのっていました。
「ののして、ののとの、うにのんっていたした。
「ののは、のっていっていっていたした。
「のいは、うっていました。
「のののののののののっ蠍揃祖爆麗ニは、っていた。
「のののののうのいのの障。」
「あのは、、のをのいののののののった。
「炉っていた。
「ののはのいくいました。
「ののは、ののはのののののいったいた。
「のののののにはってい茂した。
「ののは、、うのののの

38743/38743 [==============================] - 10s 258us/step - loss: 2.4803
epoch:  12
seed:  「ではみなさんは、そ
「ではみなさんは、そうにもったりました。
「ああ、そのどこかない。」
「ああ、そうどここでもうなったのです。
ジョバンニは、すっかりまったのです。
「ああ、そのですか。」
「いう、うかっていたのです。そのです。」
「ああ、そうに、あありもした。そのです。」
「ああ、そう、そのですから、車のですか。」
「いうもうにしているのです。
「ああ、どうどうか。」
「そうだ。」
「ああ、そう。」
「ああ、そうに、あありもいた。いたのですか。」
「ああ、そうから、いつから、きっきのように見えるとこのでした。
「ああ、そう。」
「ああ、ジョバンニは、すかからあるときなっところのから、こっちの水を云いました。
「ああ、そのです。」
「ああ、そうから、こんでものあからのようになって、その中にはたくさんだろうか。」
「ああ、そうに、あるようになって、もうかったりです。その人は、あげまた方へなって、まったのです。
「ああ、その中には

Epoch 14/60
38743/38743 [==============================] - 10s 258us/step - loss: 2.3941
epoch:  13
seed:  「ではみなさんは、そ
「ではみなさんは、そうになって、もうかりました。
「ああ、その人は、ものですから、いっぱいにないました。
「ああ、その人は、ものですから、いっぱいにないました。
「ああ、その人は、ものですから、いるのでした。
「いました。
「ああ、それはそうになって、まったりしていました。
「いました。
「ああ、どうそうちを見えるのでした。
「あの鳥捕りのように見えました。
「いうした。
「そう、その小さながいるのです。」
「ああ、それはそうにしているのでした。
「ああ、それはもうって来るのでした。
「ああ、それはそうにこのでした。
「それはもうって来る。」
「いました。
「そう、そのですか。」
「いうように、いっぱりについて来るよ。」
「そうに。」
「ああ、そうに、あるのでした。それならのでしょう。」
「ああ、そうに、あるのです。そ

38743/38743 [==============================] - 10s 261us/step - loss: 1.4993
epoch:  28
seed:  「ではみなさんは、そ
「ではみなさんは、そうにうとこのを見たと、ないともうすきにおっていました。
「どこもなもおだからないとした気がったりです。
ジョバンニは、すぐに立っていました。
「お母さん。二人は、車室の中に、水をお持ってしたがかないかしました。
ジョバンニは、
「ケットルラものれてにもったした。いると、おっとの下のたちは、どきどんないよ。」
「あああの鳥をさった。ですか。」鳥捕りは、またちの前の水のように、みんな水晶の水した。ジョバンニは、じゃくりませんからそうでした。さっきからはこんでもい。」
ジョバンニは、いうだ。あるようになって、あっている子かさんは、わずかって燃えてよるでしょうか。」
「それは、カムパネルラと、カムパネルラは、窓の外でを出しました。
「お父さんがお母さんのはなんかっちょんから光をつりして。」
「ええ、あすかになって、まだその。」
「ああ、こんなんだ。どこかでもあんながおもなく、あるとを見ました。
「

Epoch 30/60
38743/38743 [==============================] - 10s 261us/step - loss: 1.4592
epoch:  29
seed:  「ではみなさんは、そ
「ではみなさんは、そうにうとしました。そしてもうなくらみの姉弟に膝らあるのとしました。そしていとないといるとこからしたのです。
ジョバンニはそう云ってした。その銀河がはなんでした。」
「そうだ。」
「ああ、それはそうにしからいるのでした。
「ああしたりはそう、そのからはいっぱいなってカムパネルラがまっ赤くなって、もう一つずたくしたのです。
そしていました。
「それはそのからいいのいうな。」
「ああ、それはそうにしからの子供が、れたくもいっぱいにないました。

「そうだんだ。それから、いっぱいのように、きました。
「それはもうって来ると思いました。その人が見ているのでした。
「あの人はいました。
「ええ、どんで行くてしたした。その天気輪の柱の前の入にたけんなら居たのまです。」
「ああ、それはそうにしからもうして、まったり

38743/38743 [==============================] - 10s 263us/step - loss: 1.0199
epoch:  44
seed:  「ではみなさんは、そ
「ではみなさんは、そうにうとうころに行きました。もジョバンニはおっとことを云っずかにそうに思わながらしながかました。ジョバンニはまるで夢を見るよ。」
「えええのあるりでした。
「あかしはい旗をこっ、私どもどきいまでも、。」
二人は、車のですからふるといろいまだしていろいろところいろなから、お父さんなも少し水のな。」とるろが、お母さんがいるんでしょうか。」
「ぼんだり、私どもの太いくそのお父さんや、立派な銀河のへ祭にをうしてこっちにいるんやっぱり、その霧の中にをてくる光っていました。
「とな、」カムパネルラがったりですか。そら、
「いっ、ぼんやりしたら煙を赤いました。
「お母さん。二人は、二人も生けて死して、砂のましそが、トといいろうか。川がなんといろないらですって立派ながひるとい、ふこかえるのだをもうっていまいでした。
「ああ、こっちは見ているよ。」
「ああからゆれたしゃ。」
「鶴なようになりがそれから。」


Epoch 46/60
38743/38743 [==============================] - 10s 265us/step - loss: 1.0041
epoch:  45
seed:  「ではみなさんは、そ
「ではみなさんは、そうくっとこまりました。
「さや、ごらんなさい。どうかすったともうじろなからって来た。）ジョバンニが云いました。
「僕もうじろい雑だた。ジョバンニは、じゃくしていました。それはもあたまえたちうしろうって、ときました。もうとここらはクリスシャンをまったょうじろあてるれるながねて、たくさんはいるんです。こまです。ころいたちは、こら下の方の水のように見えるのように、するとことを云いました。
「もう駄目です。落ちてから四十五分たちましたから。」
「ああ、そこへ行って一つのごとごともうん。」ジョバンニは、すると手をあるまででか。」と云いました。きるとカムパネルラの形を見ました。そう、ほんとうにその大きなとうつかにもっと云いながらしは大きなへ行くて、もいくなって立っているだから、いつから来たっているのでした。
「

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

loss = history.history['loss']
plt.plot(np.arange(len(loss)), loss)
plt.show()